# Simulate a user accessing scenes to estimate # of sectors needed to unseal

## Imports

In [1]:
import random
import geopandas as gpd
from scipy.spatial import KDTree
import numpy as np

## Prepare Landsat scenes (Rows/Paths) of the US

### Fetch files

In [9]:
# From https://www.usgs.gov/media/files/landsat-wrs-2-descending-path-row-shapefile
landsat_scenes_shp = gpd.read_file("https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/WRS2_descending_0.zip")

# From https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
us_states_shp = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip")

# From https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
msa_boundaries = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_cbsa_20m.zip")

### Reproject to WGS84 (to match Landsat)

In [12]:

us_states_shp = us_states_shp.to_crs(epsg=4326)
msa_boundaries = msa_boundaries.to_crs(epsg=4326)

### Clip Scenes to Continential US 

In [ ]:
us_states_continential = us_states_shp[us_states_shp["STUSPS"].isin(["AK", "HI", "PR", "VI", "GU", "AS", "MP", "UM"]) == False]
landsat_scenes_clipped = gpd.clip(landsat_scenes_shp, us_states_continential)
print(len(landsat_scenes_clipped)) # Should be about 457 scenes

### Grab the rows and paths of the scenes

In [6]:
landsat_scenes_clipped_rows_paths = landsat_scenes_clipped[["PATH", "ROW"]]

## Prepare US Metropolitan Statistical Areas (MSAs)

### Intersect MSAs with Landsat scenes

In [16]:
msa_boundaries_intersect= gpd.sjoin(msa_boundaries, landsat_scenes_clipped, how="inner", op="intersects")
msa_boundaries_intersect_rows_paths = msa_boundaries_intersect[["PATH", "ROW"]]

/Users/jsolly.admin/Documents/code/easier/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3318: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [17]:
msa_boundaries_intersect_rows_paths

,PATH,ROW
0,26,36
243,26,36
364,26,36
506,26,36
710,26,36
...,...,...
925,41,28
934,10,29
934,12,28
934,10,28


## User Settings

In [ ]:
scenes_requested_per_request = random.randint(1, 5)
number_of_requests = 1000

## Sector and Cache Settings

In [ ]:
cache_time = 1000 # Number of requests (loops) before hot storage cache is cleared
cache_hit_count = 0
sector_scene_count = 25
sector_packing_method = "random-greedy" # "random-greedy" or "sequential"

## Pack Sectors

In [ ]:
row = range(0,10000)
path = range(0,10000)
scenes = (list(zip(row,path))) # Simulate 10K scenes with 10000 rows and 10000 paths (each scene is a row/path combination)
tree = KDTree(scenes)

car_scenes = [] # Each car will have a list of scenes that will be used to pack it. len(car_scenes) = scenes / 25
scene = random.choice(scenes)

while True: # Keep packing as long as there are still scenes left
    k_nearest_neighbors = tree.query(scene, k=25) # Pack each CAR file with 25 scenes
    car_scenes.append([scenes[i] for i in k_nearest_neighbors[1]])
    scenes = [v for i, v in enumerate(scenes) if i not in k_nearest_neighbors[1]] # Remove the scenes that were packed into the CAR file
    if not scenes:
        break
    tree = KDTree(scenes) # Rebuild tree with nearest neighbors removed
    scene = random.choice(scenes)

car_scenes

## Simulate User Access

In [ ]:
for i in range(number_of_requests):
    msa = random.choice(msa_boundaries_intersect_rows_paths.values)
    
    